In [6]:
#export
"""Nice website analyzer based on Selenium. Whenever I used Selenium
in the past, I've always have to find how to reach specific buttons,
what paragraphs to capture, things like that, all manually. That
really limits my usage of it. Recently, I've made the module ``kapi``,
capable of doing a bunch of DL stuff and feels like this is the right
time to make something on top of Selenium that is able to pull out the
main article, body, header, right/left bar automatically. After that, I
can grab the main text to put it into an embedding db to use it further
downstream. Example::

    from k1lib.imports import *
    
    browser = selen.getBrowser() # fires up the browser
    browser.get("https://en.wikipedia.org/wiki/Cheese")
    
    page = selen.Page.analyze(browser) # analyze everything. For a typical page, should take 8-14s total
    page.draw()             # run this inside a notebook to view all stages
    page.mainContents()     # return List[Element] of potential big pieces of the page, like header, footer, left/right bar, etc.

    elem = page.mainContent # grabs what it think is the most likely meats of the page, ignoring everything else
    elem.obj                # accesses the internal Selenium Element
    elem.obj.text           # grabs element's text content
    elem.obj.x              # grabs element's x location. Works with .y too
    elem.obj.w              # grabs element's width. Works with .h too
"""
import k1lib as k1, numpy as np; import k1lib.cli as cli; import k1lib.p5 as p5
from typing import List
by = k1.dep("selenium.webdriver.common.by")
__all__ = ["getBrowser", "Page", "Element"]

In [7]:
#export
# dummy classes representing type information
class Meta: pass # obj:BaseElement, x, y, w, h, wh (5), type, path, %area of parent (8)
class Browser: pass
Stages = List[List[Meta]]

In [ ]:
class ExtBrowser: # represents 
    def __init__(self):
        pass

In [279]:
#export
def getBrowser() -> Browser:
    """Launches new browser and return object to manage"""
    return k1.dep("selenium.webdriver", "selenium==4.8.3").Chrome(k1.dep("webdriver_manager.chrome", "webdriver_manager==4.0.1").ChromeDriverManager().install())
def getMeta(e:"Element") -> Meta:
    loc = e.location; size = e.size
    return [e, loc["x"], loc["y"], size["width"], size["height"], size["width"]*size["height"], None, "r", 0]
def getChildren(parent:Meta, autoInc): # tries to discover children of this element that are somewhat reasonable
    ogParent = parent
    while True:
        pw = parent[3]; ph = parent[4]
        try: ch1 = parent[0].find_elements(by.By.XPATH, "*") | cli.apply(getMeta) | cli.deref() # "ch" for children
        except: return [] # elements can go stale!
        ch2 = ch1 | cli.filt("x>2000", 5) | cli.deref(); ch3 = ch2
        if len(ch3) == 0: return [] # short circuits
        if len(ch3) > 1: break
        parent = ch3[0] # child encompass nearly all of parent, so loop around more, because it's not useful
    ch3 = ch3 | cli.apply(lambda _: f"{ogParent[7]}/{autoInc()}", 7) | cli.deref() # injects path into children
    """
    tries to figure out what type of decomposition is this. There are 3 types right now:
    - partition: clear decomposition into multiple large components
    - delegation: parent has 1 giant component, and many other small components
    - weird: children overlap other children, children bigger than parent (indicating position absolute/fixed). Temporarily not enabled"""
    px, py, pw, ph = ogParent[1:5]; pwh = pw*ph or 1e-3; pad = 10 # detect children bigger than parent
    # if not (ch3 | cut(1, 2, 3, 4) | ~apply(lambda x,y,w,h: x >= px-pad and x+w <= px+pw+pad and y >= py-pad and y+h <= py+ph+pad) | aS(all)): ogParent[6] = "weird"
    if False: pass
    else:
        pAreas = ch3 | cli.cut(5) | cli.apply(cli.op()/pwh) | ~cli.sort(None) | cli.deref() | cli.aS(np.array) # %area of children
        nAreas = pAreas / pAreas.sum() # normalized pAreas, so that all child areas add up to 1, to judge relative size of children, instead of area fractions alone
        if nAreas[0] > 0.85 and pAreas[0] > 0.7: ogParent[6] = "delegation" # >0.7 part means that if the child dominates other children but doesn't dominate the parent, then it's a partition
        else: ogParent[6] = "partition"
        for e, pA in zip(ch3, pAreas): e[8] = pA
    return ch3
def getStages(browser:Browser):
    body = getMeta(browser.find_element(by.By.CSS_SELECTOR, "body")); ch = [body]; stages = [ch]; autoInc = k1.AutoIncrement()
    for i in range(30):
        ch = ch | cli.apply(getChildren, autoInc=autoInc) | cli.joinSt() | cli.deref()
        if len(ch) == 0: break
        stages.append(ch)
    return stages
def draw(elem:Meta):
    try:
        text = elem[0].text
        p5.stroke(*([255, 0, 0] if elem[6] == "partition" else [0, 0, 0])); p5.noFill(); p5.rect(*elem[1:5])
        p5.noStroke(); p5.fill(0); p5.text((f"{elem[7]} - {elem[6]}\n" + text).split("\n") | cli.head(3) | cli.join("\n"), elem[1], elem[2]+10)
    except: pass
def drawStages(stages:Stages):
    body = stages[0][0]; imgs = []
    w = body[3]; h = body[4]
    if w < 10 or h < 10: # some sites (looking at you, YouTube), have 0px body height, which screws the rendering up
        w,h = stages | cli.joinSt() | cli.cut(1, 2, 3, 4) | ~cli.apply(lambda x,y,w,h: [x+w,y+h]) | cli.filt("x<3000", [0,1]) | cli.transpose() | cli.apply(~cli.sort(None) | ~cli.head(0.05) | cli.item() | cli.op()+20) # some site have elements way beyond bounds. So this is to filter out those elements
    w = min(w, 3000); h = min(h, 3000) # again, making sure it doesn't draw something completely insane
    for elems in stages:
        p5.newSketch(w, h, False); p5.background(255)
        elems | cli.apply(draw) | cli.deref(); imgs.append(p5.svg())
    return imgs

In [ ]:
#export
class Page:
    def __init__(self, stages:Stages):
        self.stages = stages; self._imgs = None # cached images
        self.path2Elem = stages | cli.joinSt() | cli.apply(lambda x: [x[7], x]) | cli.toDict()
        self.idx2Elem = stages | cli.joinSt() | cli.apply(lambda x: [x[7].split("/")[-1], x]) | cli.toDict()
        self.path2CPaths = stages | cli.joinSt() | cli.cut(7) | cli.apply(lambda x: ["/".join(x.split("/")[:-1]), x]) | cli.groupBy(0, True) | cli.apply(cli.item().all(), 1) | cli.deref() | cli.toDict(f=lambda:[]) # path to children paths
        self.w, self.h = self.stages[0][0][3:5]; self.wh = self.w*self.h
    @staticmethod
    def analyze(browser, bounded=True):
        """Analyze whatever is on the browser at this moment.

:param bounded: if True, adjusts all internal bounding boxes so that they don't overflow. If False, then children can be bigger than its parent"""
        page = Page(getStages(browser))
        return page.bound() if bounded else page
    def draw(self):
        """Quickly views each analysis stages, see bounding boxes, their paths and whatnot"""
        if not self._imgs: self._imgs = drawStages(self.stages)
        return self._imgs | k1.viz.Carousel()
    def __getitem__(self, s) -> "Element | None":
        s = f"{s}"
        if s in self.path2Elem: return Element(self, self.path2Elem[s])
        if s in self.idx2Elem: return Element(self, self.idx2Elem[s])
    def mainContents(self):
        """Returns some candidate Elements that seems to be the main content. You'd have
to write some minimum code to determine what you'd like to use, but the bulk
of the work by this point is done"""
        # 3 sorts: by area, by #children, by area again. Select fewer and fewer at each stage. This should weed out uninteresting elements
        a = self.stages | cli.joinSt() | cli.filt(cli.op()=="partition", 6) | ~cli.sort(5) | cli.cut(7, 5) | cli.head(10) | cli.deref()
        return a | cli.cut(0) | cli.lookup(self) | cli.deref()
        # b = a | ~apply(lambda p,a: [p,a,len(self.path2CPaths[p])]) | ~sort(2) | head(5) | ~sort(1) | cut(0) | deref()
        c = b | cli.apply(lambda path: self[path]) | cli.deref()
        return [a, b, c][mode]
    @property
    def mainContent(self):
        """Really tries to extract out the main content, so that this can be
automated. It might not be good, but at least it's automated."""
        try: return self.mainContents() | cli.filt("x.pArea < 0.7") | cli.item()
        except: return self.mainContents() | cli.item()
    def bound(self) -> "Page":
        """Some children is detached from the parent outside of it, and can grow bigger. But
that messes up with the ranking technique that I have right now. So purpose of this
is to limit the children to the parent's size"""
        stages = self.stages | cli.deref(); path2Meta = []
        for stage in stages:
            for elem in stage: path2Meta.append([elem[7], elem])
        path2Meta = path2Meta | cli.toDict()
        for stage in stages[1:]:
            for elem in stage:
                parent = path2Meta[elem[7].split("/")[:-1] | cli.join("/")]
                ex1 = elem[1]; ex2 = elem[1] + elem[3]
                ey1 = elem[2]; ey2 = elem[2] + elem[4]
                px1 = parent[1]; px2 = parent[1] + parent[3]
                py1 = parent[2]; py2 = parent[2] + parent[4]
                elem[1] = max(ex1, px1); elem[3] = min(ex2, px2) - elem[1]
                elem[2] = max(ey1, py1); elem[4] = min(ey2, py2) - elem[2]
                elem[5] = elem[3]*elem[4]
        return Page(stages)
class Element:
    def __init__(self, page, data):
        self.page = page; self.data = data
    @property
    def parent(self):
        """Grab this element's parent element. If not found, then return None"""
        return self.page["/".join(self.data[7].split("/")[:-1])]
    @property
    def obj(self): return self.data[0]
    @property
    def x(self): return self.data[1]
    @property
    def y(self): return self.data[2]
    @property
    def w(self): return self.data[3]
    @property
    def h(self): return self.data[4]
    @property
    def wh(self): return self.data[5]
    @property
    def path(self): return self.data[7]
    @property
    def children(self) -> "list[Element]": return [self.page[cp] for cp in self.page.path2CPaths[self.data[7]]]
    @property
    def pArea(self):
        """Percentage area of this element vs the entire page"""
        return self.wh/(self.page.wh)
    def __repr__(self):
        path = f"'{self.data[7]}'".ljust(30)
        x = f"{self.x}".ljust(4); y = f"{self.y}".ljust(4)
        w = f"{self.w}".ljust(4); h = f"{self.h}".ljust(4)
        type_ = f"'{self.data[6]}'".ljust(9)
        area = f"{round(self.data[8]*100)}%".rjust(4)
        area2 = f"{round(self.pArea*100)}%".rjust(4)
        nChildren = f"{len(self.children)}".ljust(4)
        return f"<Element path={path} loc=({x}, {y}) size=({w}, {h}) type={type_} %area(parent)='{area}' %area(body)='{area2}' #children={nChildren}>"

In [4]:
!../export.py selen --upload=True

/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-11-26 00:27:50,407	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2023-11-26 00:27:50,414	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
----- exportAll
12204   0   60%   
7989    1   40%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.4.4.5
Uninstalling k1lib-1.4.4.5:
  Successfully uninstalled k1lib-1.4.4.5
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/